# Treinamento

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
############## SEPARANDO ENTRE DADOS DE TREINO E TESTE ##############

# Pega as duas primeiras colunas (key, nome_arqivo) do arquivo LBP.csv
df = pd.read_csv('./data/caracteristicas/LBP.csv', usecols=[0,1])

# Remove as linhas onde há um nome_arquivo repetido
df = df.drop_duplicates(subset=['nome_arquivo'])

# Remova o elemento em que só há uma ocorrência de classe
df = df.groupby('key').filter(lambda x: len(x) > 1)

# Coloca a coluna key em uma lista chamada classes e a coluna nome_arquivo em uma lista chamada arquivos
classes = df['key'].tolist()
arquivos = df['nome_arquivo'].tolist()

# Separe quais arquivos serão usados para treino e quais serão usados para teste
arquivos_treino, arquivos_teste, classes_treino, classes_teste = train_test_split(arquivos, classes, test_size=0.2, random_state=42, stratify=classes)

# Carregar o arquivo LBP.csv
df_lbp = pd.read_csv('./data/caracteristicas/LBP.csv')

# Selecionar apenas as linhas que estão nos arquivos de treino
df_lbp_treino = df_lbp[df_lbp['nome_arquivo'].isin(arquivos_treino)]

# Selecionar apenas as linhas que estão nos arquivos de teste
df_lbp_teste = df_lbp[df_lbp['nome_arquivo'].isin(arquivos_teste)]

df_lbp_teste.head(20)

# Separa entre y_treino e X_treino
y_treino = df_lbp_treino['key'].values
X_treino = df_lbp_treino.drop(['key', 'nome_arquivo'], axis=1).values

# Separa entre y_teste e X_teste
y_teste = df_lbp_teste['key'].values
X_teste = df_lbp_teste.drop(['key', 'nome_arquivo'], axis=1).values

In [ ]:
############## NORMALIZANDO OS DADOS ##############

# Cria um objeto para normalizar os dados
scaler = StandardScaler()

# Normaliza os dados de treino
scaler.fit(X_treino)
X_treino = scaler.transform(X_treino)

# Normaliza os dados de teste
scaler.fit(X_teste)
X_teste = scaler.transform(X_teste)

In [ ]:
############## TRATANDO OS VALORES NAN ##############

# Substitui os valores NaN por 0
X_treino = np.nan_to_num(X_treino)
X_teste = np.nan_to_num(X_teste)

In [ ]:
############## TREINANDO O MODELO ##############

# Cria um objeto para o classificador SVM
svm = SVC(kernel='linear', C=1, random_state=42)

# Treina o modelo
svm.fit(X_treino, y_treino)

# Testa o modelo
y_pred = svm.predict(X_teste)

In [ ]:
############## VOTAÇÃO ##############

# copie o df df_lbp_teste para df_lbp_teste_votacao e adicione a coluna 'predicao'
df_lbp_teste_votacao = df_lbp_teste.copy()
df_lbp_teste_votacao['predicao'] = y_pred

# Faça a votação para cada nome_arquivo
df_lbp_teste_votacao = df_lbp_teste_votacao.groupby('nome_arquivo')['predicao'].agg(lambda x:x.value_counts().index[0]).reset_index()

# Crie um df com as classes reais
df_lbp_teste_real = df_lbp_teste.drop_duplicates(subset=['nome_arquivo'])

# Junte os dois df
df_lbp_teste_real = df_lbp_teste_real.merge(df_lbp_teste_votacao, on='nome_arquivo')

In [60]:
############## AVALIANDO O MODELO ##############

# Imprime o relatório de classificação
print(classification_report(df_lbp_teste_real['key'], df_lbp_teste_real['predicao']))


              precision    recall  f1-score   support

     abethr1       0.00      0.00      0.00         3
     abhori1       0.00      0.00      0.00        25
     abythr1       0.00      0.00      0.00         6
     afbfly1       0.00      0.00      0.00         4
     afdfly1       0.00      0.00      0.00         6
     afecuc1       0.00      0.00      0.00        18
     affeag1       0.00      0.00      0.00        10
     afgfly1       0.00      0.00      0.00         2
     afghor1       0.00      0.00      0.00        14
     afmdov1       0.00      0.00      0.00         7
     afpfly1       0.00      0.00      0.00        21
     afpwag1       0.00      0.00      0.00        16
     afrgos1       0.00      0.00      0.00        11
     afrgrp1       0.00      0.00      0.00         5
     afrjac1       0.00      0.00      0.00         6
     afrthr1       0.00      0.00      0.00         9
     amesun2       0.00      0.00      0.00         9
     augbuz1       0.00    

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av